In [0]:
from pyspark.sql.functions import *
from pyspark.sql import functions as func
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")

Out[1]: DataFrame[key: string, value: string]

In [0]:
#Leitura dos CSVs

dbutils.fs.ls("/mnt/blobed/")
calendario = spark.read.options(sep=";", header=True, encoding = "UTF-8").csv("/mnt/blobed/modulo_02/calendario/calendario.csv")
feriados = spark.read.options(sep=";", header=True, encoding = "UTF-8").csv("/mnt/blobed/modulo_02/calendario/feriados_nacionais.csv")
dados_mensais = spark.read.options(sep=";", header=True, encoding = "UTF-8").csv("/mnt/blobed/modulo_02/detran/mensal/2022_09_06_detranmensal.csv")
dados_semanais = spark.read.options(sep=";", header=True, encoding = "UTF-8").csv("/mnt/blobed/modulo_02/detran/semanal/2022_09_05_detransemanal.csv")

In [0]:
#Calendario - Remover Hora da Coluna Data
#Feriado - Calcular dias e preencher/ Criar Coluna de Dia útil 0/1 e Tipo (Dia útil | fim de semana | nome do feriado)

calendario = calendario.withColumn("Data", func.to_date(func.to_timestamp(col("Data"), "dd/MM/yyyy")))
feriados = feriados.na.drop(how = "any")

feriados = feriados.withColumn("Data", func.to_date(func.to_timestamp(col("Data"), "dd/MM/yyyy")))

In [0]:
#Adicionado ID para obtenção de coluna ID
calendario = calendario.select("*").withColumn("id", monotonically_increasing_id())

def diaDaSemanaNumero(colunaNum):
    if colunaNum % 7 == 3:
        return "7"
    if colunaNum % 7 == 4:
        return "1"
    if colunaNum % 7 == 5:
        return "2"
    if colunaNum % 7 == 6:
        return "3"
    if colunaNum % 7 == 0:
        return "4"
    if colunaNum % 7 == 1:
        return "5"
    if colunaNum % 7 == 2:
        return "6"
    
def diaUtil(ColunaSemanaNulls, ColunaTipo):
    if (ColunaSemanaNulls == "6" or ColunaSemanaNulls ==  "7") and ColunaTipo == None:
        return "Fim de Semana"
    elif ColunaTipo == None:
        return "Útil"
    else:
        return ColunaTipo
    
def numeroDia(ColunaUtil):    
    if ColunaUtil == "Útil":
        return "1"
    else:
        return "0"

    
diasNum = udf(diaDaSemanaNumero, StringType())
diaTeste = udf(diaUtil, StringType())
diasBinario = udf(numeroDia, StringType())

#Lembrar que 1 é segunda-feira e 7 é domingo
calendario = calendario.withColumn("Número", diasNum('id').cast('int'))


#Left Join do Calendario com Feriados
calendario = calendario.join(feriados,'Data','left')

#Drop do Dia da Semana
calendario = calendario.drop("Dia da Semana")

#Renome da coluna
calendario = calendario.withColumnRenamed("Feriado","Tipo")

#Adição de dias que não são feriados
calendario = calendario.withColumn("Tipo", diaTeste('Número','Tipo').cast('string'))

#Adição do binário
calendario = calendario.withColumn("0-1", diasBinario('Tipo').cast('int'))

#Retirada da coluna id
calendario = calendario.drop("id")

In [0]:
#Resultado
display(calendario)

Data,Número,Tipo,0-1
2016-06-30,4,Útil,1
2016-07-01,5,Útil,1
2016-07-02,6,Útil,1
2016-07-03,7,Útil,1
2016-07-04,1,Útil,1
2016-07-05,2,Útil,1
2016-07-06,3,Útil,1
2016-07-07,4,Útil,1
2016-07-08,5,Útil,1
2016-07-09,6,Útil,1


In [0]:
#Reformulação da tabela agregando as colunas Ano-Mes-Dia para formar um date
dados_mensais_final = dados_mensais.select(concat_ws('-',col('ano'),col('mes'),col('dia'))\
                         .alias('Data').cast('date'),'marca','modelo','versao','vin')


#Reformulação da tabela agregando as colunas Ano-Mes-Dia para formar um date
dados_semanais_final = dados_semanais.select(concat_ws('-',col('ano'),col('mes'),col('dia'))\
                         .alias('Data').cast('date'),'marca','modelo','versao','vin')

#Join dos dados
dados_final = dados_semanais_final.join(dados_mensais_final,['Data', 'marca', 'modelo', 'versao', 'vin'],'outer')



#Ordenação por datas
dados_final = dados_final.orderBy('Data')

display(dados_final)

Data,marca,modelo,versao,vin
2022-01-03,TOYOTA,HILUX,HILUX PICKUP 4P 4X4 2.4,8AJDB3CD6N1317518
2022-01-03,TOYOTA,HILUX,HILUX PICKUP 4P 4X4 2.4,8AJDB3CD6N1317535
2022-01-03,FORD,F-150,F 150 LARIAT SPORT 4X4 5.0,1FTFW1E53MFC13756
2022-01-03,HYUNDAI,HD 35,HD 35 LWB 2.5,KMFVA17RPNC356651
2022-01-03,HYUNDAI,VERNA,VERNA CB 1.4,LBECBACB2NW163888
2022-01-03,TOYOTA,HILUX,HILUX PICKUP 4P 4X4 2.4,8AJDB3CD6N1318118
2022-01-03,TOYOTA,YARIS,YARIS SEDAN GLI 1.5,MR2B29F32N1235728
2022-01-03,CHEVROLET,GROOVE,GROOVE LTZ 1.5,LZWADAGA7NG006144
2022-01-03,TOYOTA,YARIS,YARIS HB OTTO 1.5,MR2K29F3XN1234794
2022-01-03,CHEVROLET,N400 MAX,N400 MAX 1.5,LZWCDAGA3NC807556


In [0]:
#  Qual a soma de vendas por mês? 

#Agrupamento dos meses para quantificar vendais mensais
dados_mes = dados_final.groupBy(month(dados_final.Data).alias('Mês')).count()

#Ordenação dos dados
dados_mes = dados_mes.orderBy('Mês')

#Renomeação da coluna count
dados_mes = dados_mes.withColumnRenamed('count','Quantidade de Vendas')

display(dados_mes)

Mês,Quantidade de Vendas
1,5453
2,4604
3,5845
4,5740
5,5496
6,6272
7,5885
8,5358
9,189


In [0]:
#Qual a marca que mais vende?

#Agrupamento das marcas
marcas_vendidas = dados_final.groupBy(('marca')).count()

#Ordenação decrescente
marcas_vendidas = marcas_vendidas.orderBy(col('count').desc())

#Renomeação da coluna count
marcas_vendidas = marcas_vendidas.withColumnRenamed('count','Quantidade de Vendas')
#Renomeação da coluna marca
marcas_vendidas = marcas_vendidas.withColumnRenamed('marca','Marcas mais vendidas')

display(marcas_vendidas.head(1))

Marcas mais vendidas,Quantidade de Vendas
CHEVROLET,16772


In [0]:
#Qual a distribuição de vendas da Fiat ao longo do ano?

#Agrupamento dos fiats
fiats = dados_final.where("marca = 'FIAT'").groupby(month(dados_final.Data).alias("Mês")).count()

#Renomeação da coluna count
fiats = fiats.withColumnRenamed('count','Quantidade de Vendas')

#Ordenação por mês
fiats = fiats.orderBy("Mês")

display(fiats)

Mês,Quantidade de Vendas
1,8
2,27
3,8
4,36
5,150
6,139
7,54
8,39
9,3
